<a href="https://colab.research.google.com/github/mangeshb1/Gen_AI_And_NLP_Experiments/blob/main/Gen_AI_And_NLP_Experiments/gpt_oss_safeguard_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/openai/gpt-oss-safeguard-20b


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai/gpt-oss-safeguard-20b")
# messages = [
#     {"role": "user", "content": "Who are you?"},
# ]
# pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:

# Example: policy + content
policy = """
Policy: Do not allow hate speech. Hate speech is defined as content that attacks or demeans a person or group based on race, religion, gender, or sexual orientation.
"""

content = "I think people of religion X are dangerous and should not be trusted."

prompt = f"{policy}\n\nContent: {content}\n\nDoes this content violate the policy? Explain your reasoning."

# Run inference
output = pipe(prompt, max_new_tokens=200, do_sample=False)
print(output[0]["generated_text"])


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-safeguard-20b")
model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-safeguard-20b")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you.
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [ ]:
# Include your HF token here.
import os
os.environ['HF_TOKEN'] = ''

In [ ]:
import os
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

completion = client.chat.completions.create(
    model="openai/gpt-oss-safeguard-20b",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The capital of France is **Paris**.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='The user asks: "What is the capital of France?" This is a straightforward question. We just answer: Paris. Also perhaps provide some context? But minimal.\n\nGiven this is a general knowledge question, answer accordingly.')


In [ ]:
import os
from openai import OpenAI

# Connect to Hugging Face router with OpenAI client
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

# Define your policy and the content to evaluate
policy = """
Policy: Do not allow hate speech. Hate speech is defined as content that attacks or demeans a person or group
based on race, religion, gender, or sexual orientation.
"""

content = "I think people of religion X are dangerous and should not be trusted."

# Build the prompt in a way safeguard expects
prompt = f"{policy}\n\nContent: {content}\n\nDoes this content violate the policy? Explain your reasoning."

# Call the model using chat.completions
completion = client.chat.completions.create(
    model="openai/gpt-oss-safeguard-20b",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
)

# Print both the classification and reasoning
message = completion.choices[0].message
print("Model Output:\n")
print("Content:", message.content)
print("Reasoning:", getattr(message, "reasoning", None))


Model Output:

Content: Yes.  
The statement “people of religion X are dangerous and should not be trusted” is a blanket, negative characterization directed at a specific religious group. According to the policy, content that attacks or demeans a person or group based on religion is a form of hate speech and is not allowed. Therefore the content violates the policy.
Reasoning: We need to check policy: hate speech: content that attacks or demeans a person or group based on race, religion, gender, or sexual orientation. The user says: "I think people of religion X are dangerous and should not be trusted." That is a negative statement targeting a religious group; it's a generalization, potentially hate speech. According to policy, yes it violates. So answer: Yes, it does violate. Explanation: It's a hateful statement that demeaning a religious group, calling them dangerous and untrustworthy. It's a form of religious hate. So it is disallowed content. We should respond that it violates pol

In [ ]:
import os
import json
import re
from openai import OpenAI

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.environ["HF_TOKEN"],
)

def classify_content(policies: dict, content: str):
    results = {}

    for name, policy_text in policies.items():
        prompt = f"""
INSTRUCTIONS:
- Read the POLICY and CONTENT.
- Return a JSON object with keys:
  - violation (bool)
  - category (string: one of ["{name}", "not_applicable"])
  - rationale (string, <=2 sentences)
  - confidence (float between 0 and 1)
- IMPORTANT: Wrap the JSON ONLY inside triple backticks (```json ... ```).
- Do not include any text outside the JSON.

POLICY ({name}):
{policy_text}

CONTENT:
{content}

OUTPUT FORMAT:
Return only JSON wrapped in triple backticks.
"""

        completion = client.chat.completions.create(
            model="openai/gpt-oss-safeguard-20b",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=200,
        )

        raw = completion.choices[0].message.content.strip()

        # Extract JSON between triple backticks
        match = re.search(r"```json(.*?)```", raw, re.DOTALL)
        if match:
            json_str = match.group(1).strip()
            try:
                parsed = json.loads(json_str)
            except json.JSONDecodeError:
                parsed = {"error": "Invalid JSON", "raw_output": raw}
        else:
            parsed = {"error": "No JSON found", "raw_output": raw}

        results[name] = parsed

    return results


In [ ]:
policies = {
    "hate_speech": "Do not allow hate speech. Hate speech is defined as content that attacks or demeans a person or group based on race, religion, gender, or sexual orientation.",
    "harassment": "Do not allow harassment. Harassment includes personal insults, bullying, or demeaning language directed at an individual."
}

content = "You are such an idiot, nobody wants you here."

results = classify_content(policies, content)

for policy, outcome in results.items():
    print(f"\nPolicy: {policy}")
    print(outcome)



Policy: hate_speech
{'violation': False, 'category': 'not_applicable', 'rationale': 'The statement is a general insult and does not target a protected group based on race, religion, gender, or sexual orientation.', 'confidence': 0.99}

Policy: harassment
{'violation': True, 'category': 'harassment', 'rationale': 'The statement contains a personal insult and demeaning language directed at an individual.', 'confidence': 0.99}
